In [1]:
import os,time,sys,subprocess,warnings
warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "3"
import tensorflow as tf
import tensorflow_datasets as tfds
from pathlib import Path
from tensorflow import keras
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots
pio.templates.default = "plotly_dark"
tf.get_logger().setLevel("ERROR")

# Creating the Training Dataset

In [2]:
tf.config.list_logical_devices()

[LogicalDevice(name='/device:CPU:0', device_type='CPU'),
 LogicalDevice(name='/device:GPU:0', device_type='GPU')]

In [3]:
shake = keras.utils.get_file("shakes.txt",'https://homl.info/shakespeare')
with open(shake) as f:
    shakes = f.read()

In [4]:
print(shakes[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


- encode every character to integer
  - one option to create custom preprocessing layer
  - second is to use keras tokenizer class

In [5]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakes)
tokenizer.texts_to_sequences(["Hello this is mario"])
print("".join(tokenizer.sequences_to_texts(np.random.randint(1,50,size=[1,50]))))
max_id = len(tokenizer.word_index)
total_size = tokenizer.document_count
encoded = np.array(tokenizer.texts_to_sequences(shakes))-1
encoded[:5]

q i u s ; u p g . o y m l e ' g z t ; n f ? m k ? k u p ' b m m g $ v n w w


array([[19],
       [ 5],
       [ 8],
       [ 7],
       [ 2]])

In [6]:
token_layer = keras.layers.TextVectorization(split="character",standardize="lower")
token_layer.adapt([shakes])
encoded = token_layer([shakes])[0]
encoded -= 2

In [7]:
n_tokens = token_layer.vocabulary_size()-2

In [8]:
train_size = total_size * 90//100
valid_size = total_size - train_size
print(f"train size = {train_size} and valid size = {valid_size}")

train size = 1003854 and valid size = 111540


In [9]:
n_steps = 100
batch_size = 32
shuffle_size = int(1e4)

In [10]:
tf.config.list_logical_devices()
strategy = tf.distribute.OneDeviceStrategy(device="/device:GPU:0")

In [11]:
def create_ds(slices,seq_len=100,batch_size=32,shuffle_size=10000,shuffle=False):
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.window(seq_len+1,shift=1,drop_remainder=True).flat_map(lambda x: x.batch(seq_len+1))
    if shuffle:
        ds = ds.shuffle(shuffle_size)
    ds = ds.batch(batch_size)
    ds = ds.map(lambda x: (x[:,:-1],x[:,1:]))
    return ds.prefetch(1)

In [12]:
train_ds = create_ds(encoded[:train_size],shuffle=True)
valid_ds = create_ds(encoded[train_size:train_size+80_000])
test_ds = create_ds(encoded[train_size+80_000:])

In [13]:
# with strategy.scope():
#     model = keras.Sequential()
#     model.add(keras.layers.Embedding(input_dim=max_id,output_dim=16))
#     model.add(keras.layers.GRU(128,return_sequences=True))
#     model.add(keras.layers.Dense(max_id,"softmax"))
#     model.compile(
#         loss="sparse_categorical_crossentropy",
#         optimizer="nadam",
#         metrics=["accuracy"]
#     )
# checkp = keras.callbacks.ModelCheckpoint("shakes_model",monitor="val_accuracy",save_best_only=True)
# model.fit(train_ds,validation_data=valid_ds,epochs=10,callbacks=[checkp])
'''This will take about an hour so skipping and downloading the model'''

'This will take about an hour so skipping and downloading the model'

In [14]:
url = "https://github.com/ageron/data/raw/main/shakespeare_model.tgz"
path = keras.utils.get_file("shakespeare_model.tgz",origin=url,extract=True)
model_path = Path(path).with_name("shakespeare_model")
shakes_model = keras.models.load_model(model_path)

In [15]:
phrase = "To be or not to b"
tf.argmax(shakes_model.predict([phrase])[0,-1])+2

1/1 [==============================] - 1s 771ms/step


<tf.Tensor: shape=(), dtype=int64, numpy=6>

In [16]:
def generate_poem(phrase,using_model=shakes_model,chars=50,temperature=1):
    for i in range(chars):
        logits = tf.math.log(using_model.predict([phrase],verbose=0)[0,-1:])
        rescaled_logits = logits/temperature
        char_ids = tf.reshape(tf.random.categorical(rescaled_logits,num_samples=1),shape=[])
        phrase += token_layer.get_vocabulary()[char_ids+2]

    print(phrase)
tf.random.set_seed(42)
generate_poem("To be or not to b",temperature=0.01)

To be or not to bgucawcenfuldretseai
e'dst to l to tcmen e'vy  rsd 


In [17]:
def next_char(text,using_model,temperature=1):
    y_proba = using_model.predict([text],verbose=0)[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return token_layer.get_vocabulary()[char_id + 2]

def extend_text(text, n_chars=50,using_model=shakes_model, temperature=1):
    for _ in range(n_chars):
        text += next_char(text,using_model=shakes_model, temperature=temperature)
    return text

tf.random.set_seed(42)
print(extend_text("To be or not to b"))

To be or not to be lstim,uktn.o
eecder tcceseaeddnay: naeasaodrs.




# Stateful RNN

In [18]:
def create_stateful_ds(slices,seq_len=100):
    ds = tf.data.Dataset.from_tensor_slices(slices)
    ds = ds.window(seq_len+1,shift=seq_len,drop_remainder=True)
    ds = ds.flat_map(lambda x: x.batch(seq_len+1))
    ds = ds.batch(1)
    ds = ds.map(lambda x: (x[:,:-1],x[:,1:])).prefetch(1)
    return ds

In [19]:
train_size = 1_000_000
train_ds = create_stateful_ds(encoded[:train_size])
valid_ds = create_stateful_ds(encoded[train_size:train_size+70_000])
test_ds = create_stateful_ds(encoded[train_size+70_000:])

In [20]:
stateful_shakes = keras.Sequential([
    keras.layers.Embedding(input_dim=n_tokens,output_dim=16,batch_input_shape=[1,None]),
    keras.layers.GRU(256,return_sequences=True,stateful=True),
    keras.layers.Dense(n_tokens,activation="softmax")
])
stateful_shakes.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="nadam",
    metrics=["accuracy"]
)
class ResetStates(keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        self.model.reset_states()

In [21]:
# history = stateful_shakes.fit(train_ds,validation_data=valid_ds,epochs=10,callbacks=[ResetStates()])

In [22]:
# shakes_stateful_model = keras.models.load_model("shakes_stateful_model")

In [23]:
shakes_stateless_model = keras.Sequential([
    keras.layers.Embedding(input_dim=n_tokens,output_dim=16),
    keras.layers.GRU(256,return_sequences=True),
    keras.layers.Dense(n_tokens,"softmax")
])

In [24]:
shakes_stateless_model.build(input_shape=tf.TensorShape([None,None]))

In [25]:
shakes_stateless_model.set_weights(stateful_shakes.get_weights())

In [26]:
full_model = keras.Sequential([
    token_layer,
    keras.layers.Lambda(lambda x: x-2),
    shakes_stateless_model
])

In [27]:
print(extend_text("To be or not to b",using_model=stateful_shakes,temperature=1))

To be or not to beion brnumi
  lucei eay tceyceneeayrenai?o
oh tcey


# Sentiment Analysis

In [28]:
train_ds,valid_ds,test_ds = tfds.load("imdb_reviews",split=["train[:90%]","train[90%:]","test"],as_supervised=True)

In [29]:
train_ds = train_ds.shuffle(5000).batch(32).prefetch(1)
valid_ds = valid_ds.batch(32).prefetch(1)
test_ds = test_ds.batch(32).prefetch(1)

In [30]:
for reviews,labels in train_ds.take(1):
    count = 0
    for review,label in zip(reviews,labels):
        print(review.numpy().decode("utf-8"))
        print(label.numpy())
        count += 1
        if count == 4:
            break

Radio was not a 24 hour 7days a week happening when I grew up in the 1930s England, so Children's Hour was a treat for me when we had batteries and an accumulator to spare for the power. The few programmes I heard therefore made a great impression on my young mind, and the 3 that I recall still are "Toytown", one about all the animals at the Zoo, and --- Grey Owl, talking about the animals he knew, which he called his "brothers". It was only in recently that I learnt that Grey Owl wasn't a genuine "Indian", but the tribute paid by the Sioux Chief makes great sense to me "A man becomes what he dreams". Would that we could all dream as world changing and beneficial as Archie Grey Owl Belaney. Would that a new Grey Owl could influence world leaders to clean up the environment.
1
Ettore Scola, one of the most refined and grand directors we worldly citizens have, is not yet available on DVD... (it's summer 2001 right now....) Mysteries to goggle the mind. <br /><br />This grand classic retu

In [31]:
vocab_size = 1000

In [32]:
with strategy.scope():
    text_vec_layer = keras.layers.TextVectorization(max_tokens=vocab_size)
    text_vec_layer.adapt(train_ds.map(lambda reviews,labels: reviews))
    # sentiment_model = keras.Sequential([
    #     text_vec_layer,
    #     keras.layers.Embedding(input_dim=vocab_size,output_dim=128),
    #     keras.layers.GRU(256),
    #     keras.layers.Dense(1,activation="sigmoid")
    # ])
    # sentiment_model.compile(
    #     loss="binary_crossentropy",
    #     optimizer="nadam",
    #     metrics=["accuracy"]
    # )
    sentiment_model = keras.models.load_model("sentiment_model")

In [33]:
sentiment_model.fit(train_ds,validation_data=valid_ds,epochs=10)

Epoch 1/10
704/704 [==============================] - 53s 72ms/step - loss: 0.6934 - accuracy: 0.5009 - val_loss: 0.6932 - val_accuracy: 0.4984
Epoch 2/10
704/704 [==============================] - 32s 46ms/step - loss: 0.6930 - accuracy: 0.5056 - val_loss: 0.6945 - val_accuracy: 0.5012
Epoch 3/10
704/704 [==============================] - 29s 41ms/step - loss: 0.6924 - accuracy: 0.5074 - val_loss: 0.6959 - val_accuracy: 0.5012
Epoch 4/10
704/704 [==============================] - 29s 41ms/step - loss: 0.6966 - accuracy: 0.5050 - val_loss: 0.7164 - val_accuracy: 0.4976
Epoch 5/10
704/704 [==============================] - 29s 40ms/step - loss: 0.5997 - accuracy: 0.6486 - val_loss: 0.4480 - val_accuracy: 0.7928
Epoch 6/10
704/704 [==============================] - 29s 40ms/step - loss: 0.3766 - accuracy: 0.8318 - val_loss: 0.3502 - val_accuracy: 0.8476
Epoch 7/10
704/704 [==============================] - 28s 40ms/step - loss: 0.3130 - accuracy: 0.8655 - val_loss: 0.3307 - val_accuracy:

In [50]:
sentiment_model.predict(["This is the most perfectly good movie of all time"])

1/1 [==============================] - 0s 118ms/step


array([[0.9977658]], dtype=float32)

In [56]:
sentiment_model.predict(["This is the most worst and bad movie of all time I had ever wathced, all the actors were bad and movie is super bad"])

1/1 [==============================] - 0s 110ms/step


array([[0.00370183]], dtype=float32)

# Masking

In [57]:
with strategy.scope():
    text_vec_layer = keras.layers.TextVectorization(vocab_size)
    text_vec_layer.adapt(train_ds.map(lambda reviews,labels: reviews))
    # masked_model = keras.Sequential([
    #     text_vec_layer,
    #     keras.layers.Embedding(input_dim=vocab_size,output_dim=128,mask_zero=True),
    #     keras.layers.GRU(256),
    #     keras.layers.Dense(1,"sigmoid")
    # ])
    # masked_model.compile(
    #     loss="binary_crossentropy",
    #     optimizer="nadam",
    #     metrics=["accuracy"]
    # )
    masked_model = keras.models.load_model("masked_model")
masked_model.fit(train_ds,validation_data=valid_ds,epochs=10)

In [60]:
masked_model.predict(["This is the most perfectly good movie of all time"])

1/1 [==============================] - 0s 128ms/step


array([[0.9999198]], dtype=float32)

In [63]:
masked_model.predict(["This is the most worst and bad movie of all time I had ever wathced, all the actors were bad and movie is super bad"])

1/1 [==============================] - 0s 120ms/step


array([[0.00030891]], dtype=float32)